In [13]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from eden.converter.molecule import obabel
import networkx as nx
import pybel
import requests
import os.path

In [15]:
%%time
AID=1
#AID=2401
#DATA_DIR = '/Volumes/seagate/thesis/examples/data'
DATA_DIR = '/Users/jl/uni-freiburg/thesis/EDeN/examples/3Dmodel/data'
active_fname=DATA_DIR + '/AID%s_active.sdf'%AID
inactive_fname=DATA_DIR + '/AID%s_inactive.sdf'%AID

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 13.8 µs


In [16]:
active_fname

'/Users/jl/uni-freiburg/thesis/EDeN/examples/3Dmodel/data/AID1_active.sdf'

Functions for training and testing the model

In [19]:
import datetime, time
def train_obabel_model(pos_fname, neg_fname, data_dir,
                       model_type = "default",
                       model_fname=None, n_iter=40, active_set_size=1000, 
                       n_active_learning_iterations=3, threshold=1, train_test_split=0.7, 
                       verbose=False):
    
    from numpy.random import randint
    from numpy.random import uniform
    
    
    global_cache = {}
    
    # this will be passed as an argument to the model later on
    def pre_processor( data, **args):
        return data
    
    from eden.graph import Vectorizer
    vectorizer = Vectorizer()

    from sklearn.linear_model import SGDClassifier
    estimator = SGDClassifier(class_weight='auto', shuffle=True)

    #######3
    #create iterable from files
    ########
    from eden.converter.molecule import obabel

    iterable_pos=obabel.obabel_to_eden(pos_fname)
    iterable_neg=obabel.obabel_to_eden(neg_fname)
    from itertools import tee
    iterable_pos, iterable_pos_ = tee(iterable_pos)
    iterable_neg, iterable_neg_ = tee(iterable_neg)
    
    import time
    start = time.time()
    print('# positives: %d  # negatives: %d (%.1f sec %s)'%(sum(1 for x in iterable_pos_), sum(1 for x in iterable_neg_), time.time() - start, str(datetime.timedelta(seconds=(time.time() - start)))))
    
    iterable_pos, iterable_pos_ = tee(iterable_pos)
    iterable_neg, iterable_neg_ = tee(iterable_neg)
    
    vectorizer.fit(iterable_pos_)
    
    #split train/test
    from eden.util import random_bipartition_iter
    iterable_pos_train, iterable_pos_test = random_bipartition_iter(iterable_pos, relative_size=train_test_split)
    iterable_neg_train, iterable_neg_test = random_bipartition_iter(iterable_neg, relative_size=train_test_split)



    #make predictive model
    from eden.model import ActiveLearningBinaryClassificationModel
    model = ActiveLearningBinaryClassificationModel( pre_processor, estimator=estimator, vectorizer=vectorizer )

    #optimize hyperparameters and fit model
    pre_processor_parameters = {}
    #print "pre processor parameters: " + str(pre_processor_parameters)
    vectorizer_parameters={'complexity':[2,3,4],  
                           'discretization_size':randint(3, 100,size=n_iter),                     
                           'discretization_dimension':randint(3, 100,size=n_iter)}
    
    estimator_parameters={'n_iter':randint(5, 100, size=n_iter),
                          'penalty':['l1','l2','elasticnet'],
                          'l1_ratio':uniform(0.1,0.9, size=n_iter), 
                          'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                          'power_t':uniform(0.1, size=n_iter),
                          'alpha': [10**x for x in range(-8,-2)],
                          'eta0': [10**x for x in range(-4,-1)],
                          'learning_rate': ["invscaling", "constant", "optimal"]}

    print "calling optimizer.."
    model.optimize(iterable_pos_train, iterable_neg_train, 
                   model_name=model_fname,
                   fit_vectorizer=True,
                   n_active_learning_iterations=n_active_learning_iterations,
                   size_positive=-1,
                   size_negative=active_set_size,
                   n_iter=n_iter, cv=3, n_jobs=1, verbose=verbose,
                   pre_processor_parameters=pre_processor_parameters, 
                   vectorizer_parameters=vectorizer_parameters, 
                   estimator_parameters=estimator_parameters)
  
    #estimate predictive performance
    #model.estimate( iterable_pos_test, iterable_neg_test, cv=5 )
    # Had to change this call, estimate has no cv parameter
    model.estimate( iterable_pos_test, iterable_neg_test )
    
    return model

def test_obabel_model(fname, model_type = "default", model_fname=None):
    from eden.model import ActiveLearningBinaryClassificationModel

    model = ActiveLearningBinaryClassificationModel()
    model.load(model_fname)

    #create iterable from files
    from eden.converter.molecule import obabel
    if model_type == "default":
        iterable=obabel.obabel_to_eden(fname)
    elif model_type == "3d":
        iterable=obabel.obabel_to_eden3d(fname)
    
    predictions= model.decision_function( iterable )
        
    return predictions

Train the models

In [20]:
#%%time

model_fname=DATA_DIR + '/AID%s.model'%AID
model = train_obabel_model(active_fname, inactive_fname,
                           data_dir=DATA_DIR,
                           model_type = "default",
                           model_fname=model_fname,
                           n_iter=5, 
                           active_set_size=500, 
                           n_active_learning_iterations=0, 
                           threshold=1, 
                           train_test_split=0.8, 
                           verbose=1)


# positives: 11  # negatives: 6 (0.1 sec 0:00:00.080158)
calling optimizer..

Iteration: 1/5 (at 3.4 sec; 0:00:03.409559)
Best score (roc_auc): 1.000000 (1.000000 +- 0.000000)
Instances: 12 ; Features: 1048577 with an avg of 174 features per instance
class: 1 count:8 (0.67)	class: -1 count:4 (0.33)	

Failed iteration: 4/5 (at 53.7 sec; 0:00:53.733354)
Inappropriate argument value (of correct type).
Floating-point under-/overflow occurred at epoch #61. Scaling input data with StandardScaler or MinMaxScaler might help.
Classifier:
SGDClassifier(alpha=0.0001, class_weight='auto', epsilon=0.1, eta0=0.001,
       fit_intercept=True, l1_ratio=0.24344833358347337,
       learning_rate='optimal', loss='modified_huber', n_iter=23, n_jobs=1,
       penalty='l1', power_t=0.89609756710822086, random_state=None,
       shuffle=True, verbose=0, warm_start=False)
--------------------------------------------------------------------------------
Instances: 5 ; Features: 1048577 with an avg of 176 featur

/Users/jl/.local/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/jl/.local/lib/python2.7/site-packages/numpy/core/_methods.py:71: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Test the models: